In [1]:
from extras.fileOperations import getFileNamesInDir
from compmusic.extractors.makam import scoremetadata as md
import os
import sys
import codecs
import json
import pdb

In [2]:
txtfolder = '/Volumes/SHARED/data/SymbTr/txt'
scorename = 'hicaz--sarki--aksak--sazlar_calinir--yesari_asim_arsoy'
scorename = 'evic--turku--aksak--ayagina_giymis--'

scorefile = os.path.join(txtfolder, scorename+'.txt')

metadata = md.extract(scorefile, scorename, lyrics_sim_thres=.25)
metadata

    48.0: man   and Yan are in the same measure!
    46.0: mi   and of   are in the same measure!
    40.0: man   and Aş are in the same measure!
    20.0: man   and Yan are in the same measure!
    18.0: mi   and of   are in the same measure!
    12.0: man   and Aş are in the same measure!
    179, LYRICS_SECTION does not start on a measure: 12.27778
    263, LYRICS_SECTION does not start on a measure: 18.66667
    282, LYRICS_SECTION does not start on a measure: 20.27778
    552, LYRICS_SECTION does not start on a measure: 40.27778
    636, LYRICS_SECTION does not start on a measure: 46.66667
    655, LYRICS_SECTION does not start on a measure: 48.27778


{'metadata': {'composer': '',
  'form': 'turku',
  'makam': 'evic',
  'name': 'ayagina_giymis',
  'sections': [{'endNote': 62,
    'lyric_structure': 'INSTRUMENTAL',
    'melodic_structure': u'ARANAGME_1',
    'name': u'ARANAGME',
    'startNote': 1},
   {'endNote': 126,
    'lyric_structure': 'INSTRUMENTAL',
    'melodic_structure': u'ARANAGME_2',
    'name': u'ARANAGME',
    'startNote': 63},
   {'endNote': 162,
    'lyric_structure': 'B1',
    'melodic_structure': 'B1',
    'name': 'LYRICS_SECTION',
    'startNote': 127},
   {'endNote': 179,
    'lyric_structure': 'C1',
    'melodic_structure': 'C1',
    'name': 'LYRICS_SECTION',
    'startNote': 163},
   {'endNote': 212,
    'lyric_structure': 'D1',
    'melodic_structure': 'D1',
    'name': 'LYRICS_SECTION',
    'startNote': 180},
   {'endNote': 228,
    'lyric_structure': 'E1',
    'melodic_structure': 'E1',
    'name': 'LYRICS_SECTION',
    'startNote': 213},
   {'endNote': 263,
    'lyric_structure': 'F1',
    'melodic_structur

In [3]:
txtfolder = '/Volumes/SHARED/data/SymbTr'
[txtfiles, baseTemp, symbtrnames] = getFileNamesInDir(txtfolder, '.txt')
symbtrnames = [os.path.splitext(s)[0] for s in symbtrnames]
logfile = 'logs/log.txt'

metadataFolder = './metadata/'
metadataFiles = [os.path.join(metadataFolder, s + '.json') for s in symbtrnames]

In [ ]:
## get section metadata 

# stdout is directed to file to be manipulated later
old_stdout = sys.stdout
sys.stdout = codecs.open(logfile, 'w', "utf-8")

# loop through all scores
for f, mf, s in zip(txtfiles, metadataFiles, symbtrnames):
    if not s[0] == '.': # ignore temprary files
        print s
        metadata = md.extract(f, s)
        with open(mf, 'w') as text_file:
            json.dump(metadata, text_file, indent=4)
        
# return to old stdout
sys.stdout = old_stdout

print "Extraction complete!"

In [ ]:
prevLineWarn = False
prevNoLyricsWarn = False
warn_log = []
no_section_log = []
for line in reversed(open(logfile).readlines()):
    if 'Missing section info in lyrics' in line: # no lyrics, no need to copy
        prevNoLyricsWarn = True
        prevLineWarn = False
    elif prevNoLyricsWarn: # score with no lyrics
        if line[0] == ' ': # warning, copy
            prevNoLyricsWarn = True
            prevLineWarn = True
            no_section_log.append(line)
        else:
            prevNoLyricsWarn = False
            prevLineWarn = False
            no_section_log.append(line)
    elif line[0] == ' ': # warning, copy
        prevNoLyricsWarn = False
        prevLineWarn = True
        warn_log.append(line)
    elif prevLineWarn: # score with warning
        prevNoLyricsWarn = False
        prevLineWarn = False
        warn_log.append(line)
    else: # score wo warning
        prevNoLyricsWarn = False
        prevLineWarn = False

with open('logs/warn_log.txt', 'w') as ll:
    ll.writelines(reversed(warn_log))
    
with open('logs/no_section_log.txt', 'w') as ll:
    ll.writelines(reversed(no_section_log))
        

In [ ]:
no_section_compact = [n for n in no_section_log for c in ['seyir','aranagme','ornek-oz'] if c not in n]
no_section_compact = []
for n in no_section_log:
    if not ('seyir' in n or 'aranagme' in n or 'ornek-oz' in n or 'taksim' in n or 'etud' in n):
        no_section_compact.append(n)
    
with open('logs/no_section_compact_log.txt', 'w') as ll:
    ll.writelines(reversed(no_section_compact))

In [ ]:
from compmusic.extractors.makam import scoremetadata as md
import numpy as np
dists = np.matrix([[0, 0, 0.25, 0.25, 2, 2, 2, .5, 5],
        [0, 0, 0.25, 0.25, 2, 2, 2, .5, 5],
        [0.25, 0.25, 0, 0, 1.65, 1.65, 1.43, .55, 5],
        [0.25, 0.25, 0, 0, 1.65, 1.65, 1.43, .55, 5],
        [2, 2, 1.65, 1.65, 0, 0, .7, .1, 5],
        [2, 2, 1.65, 1.65, 0, 0, .7, .1, 5],
        [2, 2, 1.43, 1.43, .7, .7, 0, .15, 5],
        [.5, .5, .55, .55, .1, .1, .15, 0, 5],
        [.5, .5, .55, .55, .6, .6, .6, 6, 0]])

groups = md.getCliques(dists, 0.25)
print groups['exact']
print groups['similar']

md.semiotize(groups)

In [ ]:
sys.stdout = old_stdout
print s